In [1]:
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from sgp4.api import Satrec
from sgp4.ext import invjday
from astropy.time import Time
from astropy import units as u
from sgp4.conveniences import jday
from joblib import Parallel, delayed
from astropy.coordinates import ITRS
from datetime import datetime, timedelta
from astropy.coordinates import TEME, CartesianDifferential, CartesianRepresentation

In [2]:
username = 'asas4539@hanyang.ac.kr'
password = 'onsaemiro1729!!'

In [3]:
def get_tle(sat_num):
    # Space-Track API의 URL
    url = "https://www.space-track.org/ajaxauth/login"
    payload = {"identity": username, "password": password}

    session = requests.Session()
    response = session.post(url, data=payload)
    if response.status_code != 200:
        print("로그인에 실패했습니다.")
        return None
    response = session.get(
        f"https://www.space-track.org/basicspacedata/query/class/tle/NORAD_CAT_ID/{sat_num}/EPOCH/2022-12-31--2024-08-01/orderby/EPOCH%20asc/format/3le/emptyresult/show"
    )

    return response

In [4]:
def get_info(change_df):
    all_info = []
    earth_radius = 6378.137
    rad2deg = 180.0 / 3.141592653589793

    for i in range(len(change_df) - 1):
        sat_info = change_df.iloc[i]
        next_sat_info = change_df.iloc[i + 1]

        tle_1, tle_2 = sat_info['first_line'], sat_info['second_line']
        satellite = Satrec.twoline2rv(tle_1, tle_2)
        jdsatepoch = satellite.jdsatepoch
        jdsatepochfrac = satellite.jdsatepochF
        epochdatetime = list(invjday(jdsatepoch + jdsatepochfrac))
        epochdatetime[-1] = int(epochdatetime[-1])

        next_tle_1, next_tle_2 = next_sat_info['first_line'], next_sat_info['second_line']
        next_satellite = Satrec.twoline2rv(next_tle_1, next_tle_2)
        next_jdsatepoch = next_satellite.jdsatepoch
        next_jdsatepochfrac = next_satellite.jdsatepochF
        next_epochdatetime = list(invjday(next_jdsatepoch + next_jdsatepochfrac))
        next_epochdatetime[-1] = int(next_epochdatetime[-1])

        start = datetime(*epochdatetime)
        end = datetime(*next_epochdatetime)
        end += timedelta(seconds=1)
        epoch = start
        jd_lst = []
        fr_lst = []
        time_lst = []

        while (epoch < end):
            year = epoch.year
            month = epoch.month
            date = epoch.day
            hour = epoch.hour
            minute = epoch.minute
            second = epoch.second
            jd, fr = jday(year, month, date, hour, minute, second)

            time_lst.append(epoch)
            jd_lst.append(jd)
            fr_lst.append(fr)
            epoch += timedelta(minutes=1)

        e, r, v = satellite.sgp4_array(np.array(jd_lst), np.array(fr_lst))
        t_lst = Time(list(np.array(jd_lst) + np.array(fr_lst)), format='jd')
        teme_p = CartesianRepresentation(r[:, 0] * u.km, r[:, 1] * u.km, r[:, 2] * u.km)
        teme_v = CartesianDifferential(v[:, 0] * u.km / u.s, v[:, 1] * u.km / u.s, v[:, 2] * u.km / u.s)
        teme = TEME(teme_p.with_differentials(teme_v), obstime=t_lst)
        itrs_geo = teme.transform_to(ITRS(obstime=t_lst))
        locations = itrs_geo.earth_location
        geodetic_coords = locations.geodetic

        segment_df = pd.DataFrame({
            'time': time_lst,
            'x': r[:, 0], 'y': r[:, 1], 'z': r[:, 2],
            'vx': v[:, 0], 'vy': v[:, 1], 'vz': v[:, 2],
            'x_earth': teme_p.x.value, 'y_earth': teme_p.y.value, 'z_earth': teme_p.z.value,
            'vx_earth': teme_v.d_x.value, 'vy_earth': teme_v.d_y.value, 'vz_earth': teme_v.d_z.value,
            'altitude': (r[:, 0] ** 2 + r[:, 1] ** 2 + r[:, 2] ** 2) ** 0.5,
            'velocity': (v[:, 0] ** 2 + v[:, 1] ** 2 + v[:, 2] ** 2) ** 0.5,
            'apogee': [satellite.alta * earth_radius] * len(time_lst),
            'perigee': [satellite.altp * earth_radius] * len(time_lst),
            'inclination': [satellite.inclo * rad2deg] * len(time_lst),
            'eccentricity': [satellite.ecco] * len(time_lst),
            'raan': [satellite.nodeo] * len(time_lst),
            'longitude': geodetic_coords.lon.value,
            'latitude': geodetic_coords.lat.value,
            'height': geodetic_coords.height.value,
        })
        all_info.append(segment_df)

    result = pd.concat(all_info).reset_index(drop=True)
    result['time'] = pd.to_datetime(result['time'])
    result = result.set_index('time')
    result.sort_index(inplace=True)

    return result

In [5]:
number_lst = [44343, 44349, 44350, 44351, 44353, 44358, 58464, 46267, 53611, 53019, 51961, 47775, 51969, 58722, 56361,
              56289, 56783, 48018, 43823, 45246, 29349, 37265, 42691, 42984, 55841]

to_do_lst = []

In [7]:
for number in number_lst:
    print(number)
    # TLE_df 생성
    response = get_tle(number)
    tle_datas = response.text.split('\r\n')
    tle_first_lst = []
    tle_second_lst = []
    for i in range(0, len(tle_datas) - 2, 3):
        tle_first_lst.append(tle_datas[i + 1])
        tle_second_lst.append(tle_datas[i + 2])
    tle_df = pd.DataFrame({'first_line': tle_first_lst, 'second_line': tle_second_lst})
    tle_df.drop_duplicates(keep='first', inplace=True)
    tle_df.reset_index(inplace=True, drop=True)

    # change_df 생성
    change_times = []
    for i in range(len(tle_df)):
        tle_1, tle_2 = tle_df['first_line'].iloc[i], tle_df['second_line'].iloc[i]
        satellite = Satrec.twoline2rv(tle_1, tle_2)
        jdsatepoch = satellite.jdsatepoch
        jdsatepochfrac = satellite.jdsatepochF
        epochdatetime = invjday(jdsatepoch + jdsatepochfrac)
        change_times.append(epochdatetime)
    change_df = pd.DataFrame(change_times)
    change_df.columns = ['year', 'month', 'day', 'hour', 'minute', 'second']
    change_df['first_line'] = tle_df['first_line']
    change_df['second_line'] = tle_df['second_line']
    change_df['time'] = pd.to_datetime(change_df[['year', 'month', 'day', 'hour', 'minute', 'second']])
    change_df['time'].drop_duplicates(keep='first', inplace=True)

    # end point 삽입 및 day_change_idx 추출
    end_point = change_df.iloc[-1:].copy()
    end_point['month'] = 8
    end_point['day'] = 1
    end_point['hour'] = 0
    end_point['minute'] = 0
    end_point['second'] = 0
    end_point['time'] = pd.to_datetime(end_point[['year', 'month', 'day', 'hour', 'minute', 'second']])
    change_df = pd.concat([change_df, end_point], axis=0).copy()
    change_df.reset_index(inplace=True, drop=True)
    day_change_idx = list(change_df[['year', 'month', 'day']].drop_duplicates().index)

    # 병렬처리
    # seg_lst = []
    # for i in range(len(day_change_idx) - 1):
    #     hour_idx = change_df.iloc[day_change_idx[i]:day_change_idx[i + 1] + 1, :4].drop_duplicates().index
    #     seg_lst.append(change_df.iloc[hour_idx])
    # result_lst = Parallel(n_jobs=6)(delayed(get_info)(seg) for seg in tqdm(seg_lst))
    # result_df = pd.concat(result_lst).sort_values(by=['time'])
    
    result_df=get_info(change_df)
    print(f'Saving {number}...')
    result_df.to_csv(f'Database/{number}_new.csv')
    print(f'Saved {number}!\n')

42691
Saving 42691...
Saved 42691!

42984


KeyboardInterrupt: 